In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\long_term_frgn_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'long_term_frgn']  
df

,base_date,emd_cd,long_term_frgn
base_date,,,
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480


In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)    
data

,emd_cd,long_term_frgn
base_date,,
2018-01-31,50110250,427986.3363
2018-02-28,50110250,427240.8119
2018-03-31,50110250,513638.2631
2018-04-30,50110250,474601.3510
2018-05-31,50110250,395262.5483
...,...,...
2021-02-28,50130620,109325.4866
2021-03-31,50130620,118258.8891
2021-04-30,50130620,129227.7480


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50130580

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,long_term_frgn
base_date,
2018-01-31,63793.3979
2018-02-28,59906.8494
2018-03-31,101924.9313
2018-04-30,59501.2049
2018-05-31,51772.3884
2018-06-30,38234.6491
2018-07-31,22187.4164
2018-08-31,34174.0381
2018-09-30,24796.1718


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'long_term_frgn': base_date
 2018-01-31     63793.3979
 2018-02-28     59906.8494
 2018-03-31    101924.9313
 2018-04-30     59501.2049
 2018-05-31     51772.3884
 2018-06-30     38234.6491
 2018-07-31     22187.4164
 2018-08-31     34174.0381
 2018-09-30     24796.1718
 2018-10-31     33444.9520
 2018-11-30     30170.4257
 2018-12-31     86990.8990
 2019-01-31    149960.3142
 2019-02-28    144307.8812
 2019-03-31    182928.7493
 2019-04-30    122137.6721
 2019-05-31    146705.7161
 2019-06-30    115500.3186
 2019-07-31     70125.6680
 2019-08-31    144397.4699
 2019-09-30    100686.5100
 2019-10-31    124541.2908
 2019-11-30    141613.8225
 2019-12-31    146984.8958
 2020-01-31    241451.6529
 2020-02-29    184158.8423
 2020-03-31    204377.5891
 2020-04-30    149205.4634
 2020-05-31    205516.3900
 2020-06-30    116760.3477
 2020-07-31     72890.2283
 2020-08-31    101124.7882
 2020-09-30    199939.5400
 2020-10-31    104933.6543
 2020-11-30     86258.8755
 2020-12-31    223686.0655

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -3.115026
p-value : 0.025461
Critical Values :
	1%: -3.600983
	5%: -2.935135
	10%: -2.605963


In [8]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,1,0)(0,0,0)[1] intercept   : AIC=1012.156, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[1] intercept   : AIC=1007.680, Time=0.09 sec
 ARIMA(0,1,2)(0,0,0)[1] intercept   : AIC=1009.110, Time=0.06 sec
 ARIMA(0,1,3)(0,0,0)[1] intercept   : AIC=1010.300, Time=0.08 sec
 ARIMA(0,1,4)(0,0,0)[1] intercept   : AIC=1008.877, Time=0.13 sec
 ARIMA(0,1,5)(0,0,0)[1] intercept   : AIC=1010.631, Time=0.13 sec
 ARIMA(1,1,0)(0,0,0)[1] intercept   : AIC=1009.599, Time=0.05 sec
 ARIMA(1,1,1)(0,0,0)[1] intercept   : AIC=1007.304, Time=0.11 sec
 ARIMA(1,1,2)(0,0,0)[1] intercept   : AIC=1011.070, Time=0.09 sec
 ARIMA(1,1,3)(0,0,0)[1] intercept   : AIC=1012.041, Time=0.11 sec
 ARIMA(1,1,4)(0,0,0)[1] intercept   : AIC=inf, Time=0.26 sec
 ARIMA(2,1,0)(0,0,0)[1] intercept   : AIC=1008.671, Time=0.04 sec
 ARIMA(2,1,1)(0,0,0)[1] intercept   : AIC=1010.692, Time=0.07 sec
 ARIMA(2,1,2)(0,0,0)[1] intercept   : AIC=1008.764, Time=0.19 sec
 ARIMA(2,1,3)(0,0,0)[1] intercept   : AIC=1010.876, Time=0.30 sec
 ARIMA(3,1,0)(0

ARIMA(order=(1, 1, 1), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [9]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.long_term_frgn.values, order=(1,1,1))

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                             ARIMA Model Results                              
Dep. Variable:                    D.y   No. Observations:                   41
Model:                 ARIMA(1, 1, 1)   Log Likelihood                -499.149
Method:                       css-mle   S.D. of innovations          45316.127
Date:                Sat, 11 Sep 2021   AIC                           1006.297
Time:                        04:44:09   BIC                           1013.151
Sample:                             1   HQIC                          1008.793
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       2556.0327   1012.936      2.523      0.012     570.714    4541.351
ar.L1.D.y      0.4578      0.151      3.033      0.002       0.162       0.754
ma.L1.D.y     -1.0000      0.091    -11.049      0.0

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


# 예측하기

In [10]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([135392.13490273, 159036.12300319]), array([45316.126665  , 49839.65656441]), array([[ 46574.15872047, 224210.11108499],
       [ 61352.1911351 , 256720.05487127]]))
